In [5]:
import pandas as pd
import json
import re
import numpy as np
import jiwer

def preprocess(text):
    text = text.lower().replace("none", "")
    if 'z' in text:
        return ''
    
    return text

def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

def clean_text(text):
    text = text.replace('zzz', '')
    text = re.sub('[\.,?!@#$%]', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub('^\s', '', text)
    text = re.sub('\s$', '', text)
    return text
    

def parse_lb1(list_segment):
    list_text_wenet = []
    list_text_kaldi = []
    list_text_typing = []
    
    for segment in list_segment:
        seed = segment.get("seed")
        text_wenet = preprocess(segment.get("content", {}).get("extras", {}).get("classify", {}).get("text_wenet", ""))
        text_kaldi = preprocess(segment.get("content", {}).get("extras", {}).get("classify", {}).get("predict_kaldi", ""))
        text_typing = preprocess(segment.get("content", {}).get("tag", {}).get("text", ""))
        
        list_text_wenet.append(text_wenet)
        list_text_kaldi.append(text_kaldi)
        list_text_typing.append(text_typing)
    return {
        "seed": seed,
        "predict_wenet": clean_text(" ".join(list_text_wenet)),
        "predict_kaldi": clean_text(" ".join(list_text_kaldi)),
        "full_text": clean_text(" ".join(list_text_typing)),
    }

def calculate_wer(gt: str, pred: str):
    if gt:
        return jiwer.wer(gt, pred)
    else:
        return -1

In [19]:
asr_zalo_df = pd.read_csv("./asr_zalo_202306261751.csv")

column_asr_label_df = ['seed', 'file_name', 'label_url', 'full_text', 'predict_kaldi', 'predict_wenet']
asr_label_df = pd.DataFrame(columns=column_asr_label_df)

for idx, res in asr_zalo_df.iterrows():
    lc_path = res.get("lc_path", "")
    file_name = lc_path.split("/")[-1]
    
    label_url = res.get("label_url", "")
    lb1 = res["lb1"]
    if not isinstance(lb1, str) or lb1 == 'EMPTY' or len(lb1) == 0:
        continue
        
    # convert str to dict of lb1
    if isinstance(lb1, str):
        lb1 = json.loads(lb1)
    
    lb1_data = lb1['data']
   
    data = parse_lb1(lb1_data)
    res_dict = {
        "label_url": label_url,
        "file_name": file_name,
        **data
    }
    
    seed = data['seed']
    predict_wenet = data['predict_wenet']
    predict_kaldi = data['predict_kaldi']
    full_text = data['full_text']
    
    df = pd.DataFrame([[seed, file_name, label_url, full_text, predict_kaldi, predict_wenet]], columns=column_asr_label_df)
    # df = pd.DataFrame.from_dict(fres, orient='index')
    
    asr_label_df = pd.concat([asr_label_df, df])
asr_label_df  

,seed,file_name,label_url,full_text,predict_kaldi,predict_wenet
0,1470,P00471cf671cf9b91c2de.wav,https://api.zalo.ai/ailab_video/label_supplier...,mi có ăn chay trưòng cái tâm mi mới mới mới tị...,mi có ăn cháo ừ cái tâm ý mới tận lệ thôi,
0,1471,P00b3f4089931736f2a20.wav,https://api.zalo.ai/ailab_video/label_supplier...,chạy không nói nhanh nhe xe vô gần tới rồi nữa...,,
0,1472,P00c3fc7991407b1e2251.wav,https://api.zalo.ai/ailab_video/label_supplier...,đụ mẹ cái dây sạc bị hư rồi do củ sạc hả ta,,
0,1473,P00f65c483171db2f8260.wav,https://api.zalo.ai/ailab_video/label_supplier...,hello (hé lô) ừm chị coi lại nè cái toa này tr...,,
0,1474,P014bc7f5aacc409219dd.wav,https://api.zalo.ai/ailab_video/label_supplier...,kê ô kê anh thế thì em bao luật hai triệu cho ...,,
...,...,...,...,...,...,...
0,6195,asr-streaming-root-ailab-20230516131434-e5dcc0...,https://api.zalo.ai/ailab_video/label_supplier...,thế là nó bảo em lại đi làm bar(ba) hoặc là đi...,,
0,6196,asr-streaming-root-ailab-20230516131542-e0bf57...,https://api.zalo.ai/ailab_video/label_supplier...,cái này em thấy nó giống cái của mình không,,
0,6197,asr-streaming-root-ailab-20230516131606-20e46d...,https://api.zalo.ai/ailab_video/label_supplier...,giống cái của mình không dạ,,
0,6198,asr-streaming-root-ailab-20230516131610-4b1766...,https://api.zalo.ai/ailab_video/label_supplier...,làm ở ngoài thì làm kiểu gì mà lại dịch mà lại...,,


# handle google

In [20]:
result_asr_google_path = "./result_asr_google.txt"
with open(result_asr_google_path, "rb") as ifile:
    lines = ifile.readlines()
    lines = lines[0]
    dict_result_asr_google = json.loads(lines)

In [21]:
lc_path_asr_zalo = '/data4/data_link_async/label_supplier/audio_zalo/2023_03_27/P4d20ec9f81a66bf832b7.wav'
dict_result_asr_google[lc_path_asr_zalo]

''

In [22]:
asr_google_df = pd.DataFrame(columns=['file_name', 'lc_path_asr_google', 'predict_google'])

for lc_path, text_predict in dict_result_asr_google.items():
#     print(lc_path, text_predict)
    file_name = lc_path.split("/")[-1]
    text_predict = preprocess(text_predict)
    
    google_text_type = 'normal'
    # todo: ... check google_text_type
    if has_numbers(text_predict):
        google_text_type = 'normed'
    
    df = pd.DataFrame([[file_name, lc_path, text_predict, google_text_type]], columns=['file_name', 'lc_path_asr_google', 'predict_google', 'google_text_type'])
    asr_google_df = pd.concat([asr_google_df, df])

In [23]:
i = 10
print(asr_google_df['lc_path_asr_google'].values[i])
print(asr_google_df['predict_google'].values[i])

/data4/data_link_async/label_supplier/audio_zalo/2023_03_27/Pbc4cc8f2a5cb4f9516da.wav
ăn hủ tiếu đuôi heo giống như hôm qua hay gì hay là hủ tiếu thịt hủ tiếu bò viên để thịt viên thôi để bò viên thôi


# CONCAT 2 df

In [24]:
asr_label_df = asr_label_df.set_index('file_name')
asr_google_df = asr_google_df.set_index('file_name')

df_concat = pd.concat([asr_label_df, asr_google_df], axis=1)
df_concat = df_concat.reset_index()
df_concat.index.name = 'id'


df_concat['full_text'] = df_concat['full_text'].fillna("")
df_concat['predict_kaldi'] = df_concat['predict_kaldi'].fillna("")
df_concat['predict_wenet'] = df_concat['predict_wenet'].fillna("")
df_concat['predict_google'] = df_concat['predict_google'].fillna("")

In [25]:
df_concat['wer_google'] = df_concat.apply(lambda row: str(calculate_wer(row['full_text'], row['predict_google'])), axis=1)
df_concat['wer_kaldi'] = df_concat.apply(lambda row: str(calculate_wer(row['full_text'], row['predict_kaldi'])), axis=1)
df_concat['wer_wenet'] = df_concat.apply(lambda row: str(calculate_wer(row['full_text'], row['predict_wenet'])), axis=1)

In [26]:
df_asr_benchmark = df_concat[['seed', 'label_url', 'full_text',
       'predict_kaldi', 'predict_wenet', 
       'predict_google', 'wer_google', 'wer_kaldi', 'wer_wenet', 'google_text_type']]
df_asr_benchmark.to_csv("asr_benchmark.csv")
df_asr_benchmark

,seed,label_url,full_text,predict_kaldi,predict_wenet,predict_google,wer_google,wer_kaldi,wer_wenet,google_text_type
id,,,,,,,,,,
0,1470,https://api.zalo.ai/ailab_video/label_supplier...,mi có ăn chay trưòng cái tâm mi mới mới mới tị...,mi có ăn cháo ừ cái tâm ý mới tận lệ thôi,,mày gọi anh tra từ có tầm mấy,1.0,0.5,1.0,normal
1,1471,https://api.zalo.ai/ailab_video/label_supplier...,chạy không nói nhanh nhe xe vô gần tới rồi nữa...,,,chạy hâm nóng nhanh nha xe vào gần tới rồi nửa...,0.29411764705882354,1.0,1.0,normal
2,1472,https://api.zalo.ai/ailab_video/label_supplier...,đụ mẹ cái dây sạc bị hư rồi do củ sạc hả ta,,,,1.0,1.0,1.0,NaN
3,1473,https://api.zalo.ai/ailab_video/label_supplier...,hello (hé lô) ừm chị coi lại nè cái toa này tr...,,,,1.0,1.0,1.0,NaN
4,1474,https://api.zalo.ai/ailab_video/label_supplier...,kê ô kê anh thế thì em bao luật hai triệu cho ...,,,,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
4685,6195,https://api.zalo.ai/ailab_video/label_supplier...,thế là nó bảo em lại đi làm bar(ba) hoặc là đi...,,,,1.0,1.0,1.0,NaN
4686,6196,https://api.zalo.ai/ailab_video/label_supplier...,cái này em thấy nó giống cái của mình không,,,,1.0,1.0,1.0,NaN
4687,6197,https://api.zalo.ai/ailab_video/label_supplier...,giống cái của mình không dạ,,,,1.0,1.0,1.0,NaN


In [14]:
df_asr_benchmark[df_asr_benchmark['label_url'].isna()]

,seed,label_url,full_text,predict_kaldi,predict_wenet,predict_google,wer_google,wer_kaldi,wer_wenet,google_text_type
id,,,,,,,,,,


In [15]:
values = df_concat[df_concat['label_url'].isna()]['lc_path_asr_google'].values
dataset_name = []
for value in values:
    s = value.split('/')[-2]
    dataset_name.append(s)
set(dataset_name)
# values

set()

In [16]:
df_asr_benchmark.google_text_type.value_counts()

google_text_type
normal    1852
normed     477
Name: count, dtype: int64